In [1]:
import math
import pandas as pd
import numpy as np


import warnings
warnings.simplefilter('ignore')

## 데이터 불러오기

In [2]:
df = pd.read_csv('data/aug_train.csv')
df

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [3]:
df.shape

(19158, 14)

### enrollee_id, city 컬럼 제거

In [4]:
df.drop(['enrollee_id','city'],axis=1,inplace=True)

In [5]:
df.shape

(19158, 12)

## Nan값 처리

### gender : Nan값 처리 (삭제)
- Other은 삭제

In [6]:
df['gender'].unique()

array(['Male', nan, 'Female', 'Other'], dtype=object)

In [7]:
df['gender'].replace('Other', np.NaN, inplace=True)

In [8]:
df['gender'].unique()

array(['Male', nan, 'Female'], dtype=object)

In [9]:
# Nan값 처리
df = df.dropna(subset=['gender'])

In [10]:
df['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [11]:
df.shape

(14459, 12)

### major_discipline : Other로 넣기

In [12]:
df['major_discipline'].unique()

array(['STEM', nan, 'Arts', 'Humanities', 'Business Degree', 'No Major',
       'Other'], dtype=object)

In [13]:
df['major_discipline']=df['major_discipline'].fillna('Other')

In [14]:
df.shape

(14459, 12)

In [15]:
df['major_discipline'].isnull().sum()

0

In [16]:
df['major_discipline'].unique()

array(['STEM', 'Other', 'Arts', 'Humanities', 'Business Degree',
       'No Major'], dtype=object)

### enrolled_university : no_enrollment로 넣기

In [17]:
df['enrolled_university'].unique()

array(['no_enrollment', nan, 'Part time course', 'Full time course'],
      dtype=object)

In [18]:
df['enrolled_university']=df['enrolled_university'].fillna('no_enrollment')

In [19]:
df.shape

(14459, 12)

In [20]:
df['enrolled_university'].isnull().sum()

0

In [21]:
df['enrolled_university'].unique()

array(['no_enrollment', 'Part time course', 'Full time course'],
      dtype=object)

### education_level : 최빈값 처리

In [22]:
df['education_level'].unique()

array(['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School'],
      dtype=object)

In [23]:
# 최빈값 확인
df['education_level'].mode()

0    Graduate
dtype: object

In [24]:
df['education_level']=df['education_level'].fillna('Graduate')

In [25]:
df['education_level'].isnull().sum()

0

In [26]:
df.shape

(14459, 12)

In [27]:
df['education_level'].unique()

array(['Graduate', 'Masters', 'High School', 'Phd', 'Primary School'],
      dtype=object)

### experience 값 처리
- '>20' => 21
- '<1'  =>  1

In [28]:
df['experience'].unique()

array(['>20', '15', '5', '13', '7', '2', '16', '4', '11', '<1', '18',
       '19', '12', '10', '1', '3', '9', '6', '14', '8', '20', '17', nan],
      dtype=object)

In [29]:
df['experience'].replace('<1',1, inplace=True)

In [30]:
df['experience'].replace('>20',21, inplace=True)

In [31]:
df = df.dropna(subset=['experience'])

In [32]:
df['experience'].isnull().sum()

0

In [33]:
# int형으로 변경
df = df.astype({'experience': 'int'})

In [34]:
df['experience'].unique()

array([21, 15,  5, 13,  7,  2, 16,  4, 11,  1, 18, 19, 12, 10,  3,  9,  6,
       14,  8, 20, 17])

In [35]:
df.shape

(14446, 12)

### company_type : 
- New의 조건 (last_new_job == 'never' and company_type = Nan)

그 외에 Nan값은 제거 

In [36]:
df['company_type'].unique()

array([nan, 'Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other',
       'Public Sector', 'NGO'], dtype=object)

In [37]:
# 조건문을 사용하기 위해 Nan값을 문자열 'nan' 값으로 지정
df['company_type'].replace(np.NaN,'nan', inplace=True)

In [38]:
for i in range(0,len(df)):
    if (df['company_type'].iloc[i] == 'nan') and (df['last_new_job'].iloc[i] == 'never'):
        df['company_type'].replace('nan','New', inplace=True)

In [39]:
# 'nan'값 nan으로 변경
df['company_type'].replace('nan', np.NaN, inplace=True)

In [40]:
df = df.dropna(subset=['company_type'])

In [41]:
df['company_type'].isnull().sum()

0

In [42]:
df['company_type'].unique()

array(['New', 'Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other',
       'Public Sector', 'NGO'], dtype=object)

In [43]:
df.shape

(14446, 12)

### company_size : 
- 0의 조건 (company_type = 'New')

그 외에 Nan값은 제거 

In [44]:
df['company_size'].unique()

array([nan, '50-99', '<10', '5000-9999', '1000-4999', '10/49', '100-500',
       '10000+', '500-999'], dtype=object)

In [45]:
# 조건문을 사용하기 위해 Nan값을 문자열 'nan' 값으로 지정
df['company_size'].replace(np.NaN,'nan', inplace=True)

In [46]:
for i in range(0,len(df)):
    if (df['company_type'].iloc[i] == 'New'):
        df['company_size'].replace('nan','0', inplace=True)

In [47]:
# 'nan'값 nan으로 변경
df['company_size'].replace('nan', np.NaN, inplace=True)

In [48]:
df = df.dropna(subset=['company_size'])

In [49]:
df['company_size'].isnull().sum()

0

In [50]:
df['company_size'].unique()

array(['0', '50-99', '<10', '5000-9999', '1000-4999', '10/49', '100-500',
       '10000+', '500-999'], dtype=object)

In [51]:
df.shape

(14446, 12)

### last_new_job (문자열)
- 'never' => 0
- '>4'    => 5

In [52]:
df['last_new_job'].unique()

array(['1', '>4', '4', '3', 'never', '2', nan], dtype=object)

In [53]:
df = df.dropna(subset=['last_new_job'])

In [54]:
df['last_new_job'].replace('never', 0, inplace=True)

In [55]:
df['last_new_job'].replace('>4', 5, inplace=True)

In [56]:
df = df.astype({'last_new_job': 'int'})

In [57]:
df['last_new_job'].isnull().sum()

0

In [58]:
df['last_new_job'].unique()

array([1, 5, 4, 3, 0, 2])

## 데이터 csv로 저장

In [59]:
df.to_csv('aug_train_1.csv', index=False)

In [61]:
print(df.dtypes)

city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                  int32
company_size               object
company_type               object
last_new_job                int32
training_hours              int64
target                    float64
dtype: object


In [63]:
df.isnull().sum()

city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64